In [1]:
import os

path          = "./documents/"
doc_name_list = os.listdir(path)

sentences = []
documents = []
for doc_name in doc_name_list:
    with open(path + doc_name) as f:
        
        if "JHEP" in doc_name:
            doc_type = "JHEP"
        elif "PhysRevD" in doc_name:
            doc_type = "PhysRevD"
        else:
            doc_type = "Others"

        
        paragraphs = f.read().split("\n\n")
        text = ""
        for i, paragraph in enumerate(paragraphs):
            p = paragraph.lstrip("\n").replace("\n", " ").replace("\u3000", "").rstrip("。").replace("：", ":")

            if i == 0:
                title = p
            else:
                text = f"{title}" + " " \
                     + f"段落{i}:" + p

                sentences.append(text)

                document = {}
                document['path']  = path + doc_name
                document['type']  = doc_type
                document['title'] = title
                document['text']  = text

                documents.append(document)

documents

[{'path': './documents/JHEP03(2024)020.txt',
  'type': 'JHEP',
  'title': 'Title: Entanglement and Rényi entropies of (1+1)-dimensional O(3) nonlinear sigma model with tensor renormalization group',
  'text': 'Title: Entanglement and Rényi entropies of (1+1)-dimensional O(3) nonlinear sigma model with tensor renormalization group 段落1:Autors: Xiao Luo(Tsukuba U., Inst. Appl. Phys.), Yoshinobu Kuramashi(Tsukuba U., CCS)'},
 {'path': './documents/JHEP03(2024)020.txt',
  'type': 'JHEP',
  'title': 'Title: Entanglement and Rényi entropies of (1+1)-dimensional O(3) nonlinear sigma model with tensor renormalization group',
  'text': 'Title: Entanglement and Rényi entropies of (1+1)-dimensional O(3) nonlinear sigma model with tensor renormalization group 段落2:Date: Aug 5, 2023'},
 {'path': './documents/JHEP03(2024)020.txt',
  'type': 'JHEP',
  'title': 'Title: Entanglement and Rényi entropies of (1+1)-dimensional O(3) nonlinear sigma model with tensor renormalization group',
  'text': 'Title: E

In [2]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")
vectors = embedding_model.encode(sentences)

In [3]:
vectors.shape

(39, 1024)

In [4]:
from pymilvus import MilvusClient

client = MilvusClient("./vectorDB/test.db")


/home/luoxiao/miniconda3/envs/huggingface/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [ ]:
#client.drop_collection(collection_name="collection_test")

In [ ]:
client.create_collection(
    collection_name = "collection_test",
    dimension       = vectors.shape[1]
    )

data = [
    {
        "id"    : i, 
        "vector": vectors[i], 
        "path"  : document["path"], 
        "type"  : document["type"], 
        "title" : document["title"],
        "text"  : document["text"]
     } for i, document in enumerate(documents) 
    ]
res = client.insert(
    collection_name = "collection_test",
    data            = data
    )
res

{'insert_count': 39, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38], 'cost': 0}

In [16]:
keyword = "Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group"

results = client.search(
    collection_name = "collection_test",
    data            = [embedding_model.encode(keyword)],
    limit           = 10,
    output_fields   = ["path", "type", "title","text"]
)

for result in results:
    ids = []
    for topi in range(len(result)):
        id = result[topi]['id']
        ids.append(id)
        #print(result[topi])
        
results

data: [[{'id': 11, 'distance': 0.9251211881637573, 'entity': {'path': './documents/JHEP07(2025)036.txt', 'type': 'JHEP', 'title': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group', 'text': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group 段落5:e-Print: 2503.05144 [hep-lat]'}}, {'id': 10, 'distance': 0.9058234691619873, 'entity': {'path': './documents/JHEP07(2025)036.txt', 'type': 'JHEP', 'title': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group', 'text': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group 段落4:Published: Jul 2, 2025'}}, {'id': 12, 'distance': 0.900676965713501, 'entity': {'path': './documents/JHEP07(2025)036.txt', 'type': 'JHEP', 'title': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model wi

In [17]:
results = client.query(
    collection_name = "collection_test",
    output_fields   = ["text"],
    ids             = ids
)

for result in results:
    print(result)

{'id': 7, 'text': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group 段落1:Authors: Xiao Luo(Tsukuba U. (main)), Yoshinobu Kuramashi(Tsukuba U., CCS)'}
{'id': 8, 'text': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group 段落2:Date: Mar 7, 2025'}
{'id': 9, 'text': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group 段落3:Published in: JHEP 07 (2025) 036'}
{'id': 10, 'text': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group 段落4:Published: Jul 2, 2025'}
{'id': 11, 'text': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization group 段落5:e-Print: 2503.05144 [hep-lat]'}
{'id': 12, 'text': 'Title: Critical endpoints of three-dimensional finite density SU(3) spin model with tensor renormalization gr

In [5]:
import socket
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(("localhost", 11451))

import json
send_msg = {"query":"Tell me about XIAO LUO.", 
            "enable_thinking":True, 
            "enable_rag":True}
send_msg = json.dumps(send_msg)
send_size = int.to_bytes(len(send_msg), 4)
client.send(send_size)
client.send(send_msg.encode("utf-8"))

recv_size = int.from_bytes(client.recv(4))
recv_msg = client.recv(recv_size)
recv_msg = json.loads(recv_msg)



In [6]:
print(recv_msg["answer"])

思考:
<think>
Okay, the user asked about Xiao Luo. Let me check the provided search results.

Looking through the documents, there are multiple entries related to Xiao Luo. The first document mentions his work on entanglement and Rényi entropies of an O(3) nonlinear sigma model with tensor renormalization group. The second document talks about a study on the tensor renormalization group for the same model with and without finite chemical potential. There's also a paper on quantum phase transitions and another on critical endpoints of a spin model. The last one is about the tensor renormalization group approach to a SU(2) chiral model.

So, Xiao Luo has been researching different aspects of the O(3) nonlinear sigma model using tensor renormalization group methods. The documents show his contributions across various topics, including entanglement entropy, phase transitions, and critical points. The user might be interested in his research contributions, so I should summarize his work in th